#### Processing the data for rcmip_sspxx_emissions_1750_to_2500.csv files

Starting with the FAIR repository here: https://github.com/njleach/FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba and noting the specific commit, I used the `data_retrieval` scripts to obtain emissions data

In [ ]:
os.chdir("fair/scripts")
import data_retrieval

for ssp in ["ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]:
    df = data_retrieval.RCMIP_to_FaIR_input_emms(ssp)
    filename = "rcmip_" + ssp + "_emissions_1750_to_2500_python.csv"
    df.to_csv("../data/" + filename)

I then linearly interpolated between future decadal years as indicated by the FAIR scripts using Julia.

In [ ]:
using DataFrames, CSVFiles, Query, Interpolations

for ssp in ["ssp119", "ssp126", "ssp245", "ssp370", "ssp585"]
    df = load(joinpath(@__DIR__, "rcmip_$(ssp)_emissions_1750_to_2500_python.csv"), skiplines_begin = 6) |> DataFrame
    titles = names(df)
    titles[1] = "year"
    rename!(df, titles)

    for col in names(df)[2:end]
        idxs = (!ismissing).(df[:,col])
        itp = LinearInterpolation(df[:, :year][idxs], df[:,col][idxs])
        df[:,col] .= itp[df.year]
    end

    df |> save(joinpath(@__DIR__, "..", "rcmip_$(ssp)_emissions_1750_to_2500.csv"))
end

# add headers to file afterwards
# File Description: RCMIP SSP119 emissions scenario from 1750-2500.
# "Code Source: Extracted using default Python model version of FAIR2.0, available at https://github.com/njleach/
# FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba"
# "Paper Reference: Leach et al. 2021. ""FaIRv2.0.0: a generalized impulse response model for climate
# uncertainty and future scenario exploration,"" Geoscientific Model Development. https://doi.org/10.5194/gmd-14-3007-2021"

Note doing this work on June 6, 2023 yieled small changes for some gases for ssp585 from previous files.  The gases involved were `ch2cl, chcl3, methyl_bromine, methyl_chlorine, so2, nox, co, nnvoc, bc, nh3, oc`

### default_gas_cycle_parameters.csv

Starting with the FAIR repository here: https://github.com/njleach/FAIR/tree/47c6eec031d2edcf09424394dbb86581a1b246ba and noting the specific commit, we use the following Python script and reformat (transform) usnig Excel.

In [ ]:
gas_parameters = get_gas_parameter_defaults()
gas_parameters.to_csv("../../data/default_gas_cycle_parameters.csv")